## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-24-10-48-45 +0000,bbc,Fifty children escape after mass school abduct...,https://www.bbc.com/news/articles/cpwk5qzzppzo...
1,2025-11-24-10-48-07 +0000,bbc,What can nervous businesses expect from the Bu...,https://www.bbc.com/news/articles/c1dz7gr042no...
2,2025-11-24-10-33-05 +0000,bbc,Everything you need to know about the Budget,https://www.bbc.com/news/articles/cj4w44w42j5o...
3,2025-11-24-10-30-39 +0000,nyt,Images From Vietnam’s Year of Deadly Wet Weather,https://www.nytimes.com/2025/11/24/world/asia/...
4,2025-11-24-10-30-00 +0000,wsj,Goodwill Thrift Shops Are Getting More Glamorous,https://www.wsj.com/real-estate/commercial/goo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
73,trump,20
49,ukraine,19
31,new,16
177,plan,14
50,peace,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
182,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...,104
50,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,94
49,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,81
155,2025-11-23-17-38-05 +0000,nypost,Zelensky thanks Trump for support after critic...,https://nypost.com/2025/11/23/world-news/zelen...,74
104,2025-11-23-23-12-21 +0000,nypost,Leaked EU peace deal calls for Ukraine to keep...,https://nypost.com/2025/11/23/world-news/leake...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
182,104,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...
79,56,2025-11-24-02-21-21 +0000,nypost,New ‘NYC Noise’ app allows New Yorkers to poli...,https://nypost.com/2025/11/23/us-news/new-nyc-...
142,36,2025-11-23-19-31-39 +0000,bbc,Israel kills top Hezbollah official in first a...,https://www.bbc.com/news/articles/cn81j54xjx1o...
123,30,2025-11-23-21-41-00 +0000,nypost,Glam former prime minister Sanna Marin complai...,https://nypost.com/2025/11/23/world-news/glam-...
198,29,2025-11-23-12-00-00 +0000,wsj,While President Trump and New York Mayor-elect...,https://www.wsj.com/economy/everyone-is-talkin...
95,28,2025-11-24-00-15-23 +0000,nypost,Florida homeowner turns tables on home invader...,https://nypost.com/2025/11/23/us-news/homeowne...
116,27,2025-11-23-22-28-38 +0000,nypost,Married ex-NFL player who now heads athletics ...,https://nypost.com/2025/11/23/us-news/ex-vikin...
167,24,2025-11-23-16-49-58 +0000,cbc,"Carney, Modi agree to launch negotiations on n...",https://www.cbc.ca/news/politics/carney-modi-n...
77,22,2025-11-24-02-27-21 +0000,nypost,Long Island home health aide accused of stabbi...,https://nypost.com/2025/11/23/us-news/long-isl...
94,22,2025-11-24-00-18-33 +0000,nypost,Trio disrupts NYC’s Met Opera in wild anti-fas...,https://nypost.com/2025/11/23/us-news/trio-dis...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
